In [90]:
import pandas as pd
import requests
import json
import numpy as np
from datetime import date
from pandas.tseries.offsets import BDay

In [91]:
resp = requests.get(url='http://cotacoes.economia.uol.com.br/ws/asset/stock/list')
all_stocks = resp.json()['data']

def get_stock_id(code):
    """ Get the stock id using its code """
    for stock in all_stocks:
        if stock['code'] == '%s.SA' % code:
            return stock['idt']
    raise Exception('invalid code %s' % code)
    
def get_stock_line(code):
    """ Returns the printable version of a stock, showing its current price and daily variation """
    id = get_stock_id(code)
    url = 'http://cotacoes.economia.uol.com.br/ws/asset/%d/intraday' % id
    resp = requests.get(url=url)
    data = resp.json()
    current = data['data'][0]['price']
    return current

token = 'https://arquivos.b3.com.br/api/download/requestname?fileName=InstrumentsConsolidated&date={}&recaptchaToken='.format(date.today().strftime("%Y-%m-%d"))
#token = 'https://arquivos.b3.com.br/api/download/requestname?fileName=InstrumentsConsolidated&date=2020-06-26&recaptchaToken='
r = requests.get(token)
vJson = json.loads(r.content)
vJson['token']
optionList = 'https://arquivos.b3.com.br/api/download/?token={}'.format(vJson['token'])
r1 = requests.get(optionList)

data = r1.content.decode("ISO-8859-1").splitlines()
df = pd.DataFrame(data) 
df1=df.copy()
df1.dropna(inplace = True) 
df1 = df1.applymap(str)
df1 = pd.DataFrame(df1)
df1 = df1[0].str.split(";", n = 100, expand = True)
df1.columns = df1.iloc[0]
df1 = df1[1:]
df1 = df1.reset_index()
df1.drop("index", axis = 1, inplace=True)

df2 = df1[["TckrSymb", "Asst", "SgmtNm", "SctyCtgyNm", "XprtnDt", "TradgEndDt", "OptnTp", "ExrcPric", "OptnStyle"]]
df2 = df2[df2['SctyCtgyNm']=='OPTION ON EQUITIES']
df2['ExrcPric'] = [x.replace(',', '.') for x in df2['ExrcPric']]
df2['ExrcPric'] = df2['ExrcPric'].astype(float)

lastBD = pd.datetime.today() - BDay(1)
lastBD = lastBD.strftime("%Y-%m-%d")
token = 'https://arquivos.b3.com.br/api/download/requestname?fileName=TradeInformationConsolidated&date={}'.format(lastBD)
#token = 'https://arquivos.b3.com.br/api/download/requestname?fileName=InstrumentsConsolidated&date=2020-06-26&recaptchaToken='
r = requests.get(token)
vJson = json.loads(r.content)
vJson['token']
optionList = 'https://arquivos.b3.com.br/api/download/?token={}'.format(vJson['token'])
r1 = requests.get(optionList)

data1 = r1.content.decode("ISO-8859-1").splitlines()
dfRaw2 = pd.DataFrame(data1) 
df_trade=dfRaw2.copy()
df_trade.dropna(inplace = True) 
df_trade = df_trade.applymap(str)
df_trade = pd.DataFrame(df_trade)
df_trade = df_trade[0].str.split(";", n = 100, expand = True)
df_trade.columns = df_trade.iloc[0]
df_trade = df_trade[1:]
df_trade = df_trade.reset_index()
df_trade.drop("index", axis = 1, inplace=True)
df_trade = df_trade[(df_trade['SgmtNm']=='EQUITY PUT') | (df_trade['SgmtNm']=='EQUITY CALL')]
df_trade=df_trade[(df_trade['TradQty']!='')]
df_trade["Strike"]=0.0
df_trade["Ticker"]=""
df_trade["Validade"]=""
df_trade["Tipo"]=""
df_trade["Price"] = 0.0

for i in df_trade.TckrSymb:
    try:
        df_trade.ix[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Strike'] = df2.loc[df2["TckrSymb"]==i].ExrcPric.values[0]
        df_trade.ix[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Ticker'] = df2.loc[df2["TckrSymb"]==i].Asst.values[0]
        df_trade.ix[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Validade'] = df2.loc[df2["TckrSymb"]==i].XprtnDt.values[0]
        df_trade.ix[df_trade.loc[df_trade['TckrSymb']==i].index.values[0],'Tipo'] = df2.loc[df2["TckrSymb"]==i].OptnStyle.values[0]
    except:
        pass
        
        
df_List = df_trade[['Ticker','Tipo','SgmtNm','TckrSymb','Validade','Strike','TradQty']]
df_List=df_List[df_List["Ticker"]!=""]
vTopacoes=['ABEV3','B3SA3','BBAS3','BBDC4','BBSE3','BPAC11','BRKM5','BTOW3','COGN3','CPLE6','EGIE3','ENBR3','EVEN3','EZTC3','FLRY3','GGBR4','GNDI3','GOAU4','GRND3','GUAR3','HAPV3','HGTX3','ITSA4','ITUB4','LAME3','LAME4','LCAM3','LOGN3','LREN3','MDIA3','MGLU3','MULT3','NTCO3','ODPV3','PCAR3','PETR4','PRIO3','QUAL3','RADL3','RAIL3','RENT3','SANB11','SAPR11','SBSP3','SMTO3','STBP3','SULA11','SUZB3','TIMP3','TOTS3','VALE3','VIVT4','VVAR3','WEGE3','YDUQ3']
vTopacoes = pd.DataFrame(vTopacoes)
df_List["Filtro"]=df_List.Ticker.isin(vTopacoes[0]).astype(int)

df_List=df_List[df_List["Filtro"]==1]
df_List.drop("Filtro", axis = 1, inplace=True)

vTopacoes["Atual"] = ""
for i in vTopacoes[0]:
    vTopacoes.ix[vTopacoes.loc[vTopacoes[0]==i].index.values[0],'Atual'] = get_stock_line(i)

for i in df_trade.Ticker:
    try:
        df_trade.ix[df_trade.loc[df_trade['Ticker']==i].index.values,'Price'] = vTopacoes.loc[vTopacoes[0]==i].Atual.values[0]
        #df_trade.ix[df_trade.loc[df_trade['Ticker']==i].index.values[0],'Price'] = vTopacoes.loc[vTopacoes[0]==i].Atual.values[0]
    except:
        pass

C:\Users\kiyo_\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
C:\Users\kiyo_\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
C:\Users\kiyo_\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
C:\Users\kiyo_\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: FutureWarning: 
.ix is depre

In [93]:
vTopacoes

,0,Atual
0,ABEV3,14.29
1,B3SA3,55.44
2,BBAS3,32.36
3,BBDC4,20.96
4,BBSE3,27.75
5,BPAC11,77.27
6,BRKM5,23.55
7,BTOW3,105.89
8,COGN3,6.59
9,CPLE6,61.34


C:\Users\kiyo_\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
